In [69]:
from clickhouse_driver import Client
from datetime import datetime

host='rc1b-9sj53dbwyy94t0mx.mdb.yandexcloud.net'
port='8443'
db='rw'
user='rw_user'
password='RWdata256'
ca_cert_path=r"C:\Users\user1676\Работа\Clickhouse SSL key\CA.pem"

ch_client = Client(host, user=user, password=password, database=db, secure=True, ca_certs=ca_cert_path)

In [2]:
ch_client.query_dataframe(
    '''SELECT max(date)
FROM rw.WEB_abtests_in
'''
)

,max_date_
0,2022-09-04


In [4]:
ch_client.execute(
    '''SELECT max(date)
FROM rw.WEB_abtests_in
'''
)

[(datetime.date(2022, 9, 4),)]

In [70]:
import pandas as pd

In [119]:
def parse_exp_start_from_expId(expId_df, expId_column_name='experimentID', split_char='_', 
        date_position_in_expId=2, date_format_in_expId='%Y%m%d', final_date_format='%Y-%m-%d'):
        expId_df['exp_start_date'] = pd.to_datetime(
                expId_df[expId_column_name].str.split(split_char).str[date_position_in_expId], 
                format=date_format_in_expId).dt.strftime( final_date_format)
        return expId_df

def get_current_tests(clickhouse_client, table_name, expId_column_name='experimentID', date_column_name='date', date='yesterday()'):
    try:
        datetime.datetime.strptime(date, "%Y-%m-%d")
        current_tests = clickhouse_client.query_dataframe(f'''
        SELECT
            DISTINCT {expId_column_name}
        FROM {table_name}
        WHERE {date_column_name}='{date}'
        ''')
        return parse_exp_start_from_expId(current_tests)
    except ValueError:
        current_tests = clickhouse_client.query_dataframe(f'''
        SELECT
            DISTINCT {expId_column_name}
        FROM {table_name}
        WHERE {date_column_name}={date}
        ''')
        return parse_exp_start_from_expId(current_tests)


In [107]:
get_current_tests(ch_client, 'rw.WEB_abtests_in', 'experimentID', date='2022-09-04')

,experimentID,exp_start_date
0,dc_mobilemenu_20220831_231312889,2022-08-31
1,dc_cookiesblock2_20220816_231312889,2022-08-16
2,dc_deliverytimermobile_20220825_231312889,2022-08-25
3,rko_testpagetarrifs_20220729_227240298,2022-07-29
4,rko_vigetindentmb_20220816_227240298,2022-08-16
5,dc_cookiesblockver3_20220817_231312889,2022-08-17
6,rko_testcommunicationnul_20220803_227240298,2022-08-03
7,dc_kukiblock_20220810_231312889,2022-08-10
8,dc_ctaalfacard2_20220726_183996781,2022-07-26
9,dc_deliverytimerdesktop_20220824_231312889,2022-08-24


In [22]:
current_tests = get_current_tests(ch_client, 'rw.WEB_abtests_in', 'experimentID', date='2022-09-04')

In [31]:
import pandas as pd

In [33]:
current_tests['exp_start_date'] = pd.to_datetime(current_tests['experimentID'].str.split('_').str[2], format='%Y%m%d').dt.strftime("%Y-%m-%d")

In [34]:
current_tests

,experimentID,exp_start_date
0,dc_mobilemenu_20220831_231312889,2022-08-31
1,dc_cookiesblock2_20220816_231312889,2022-08-16
2,dc_deliverytimermobile_20220825_231312889,2022-08-25
3,rko_testpagetarrifs_20220729_227240298,2022-07-29
4,rko_vigetindentmb_20220816_227240298,2022-08-16
5,dc_cookiesblockver3_20220817_231312889,2022-08-17
6,rko_testcommunicationnul_20220803_227240298,2022-08-03
7,dc_kukiblock_20220810_231312889,2022-08-10
8,dc_ctaalfacard2_20220726_183996781,2022-07-26
9,dc_deliverytimerdesktop_20220824_231312889,2022-08-24


In [27]:
datetime.strptime('20220729', "%Y%m%d").strftime("%Y-%m-%d")

'2022-07-29'

In [ ]:
current_tests[]

In [37]:
def parse_exp_start_from_expId(expId_df, expId_column_name='experimentID', split_char='_', 
        date_position_in_expId=2, date_format_in_expId='%Y%m%d', final_date_format='%Y-%m-%d'):
        expId_df['exp_start_date'] = pd.to_datetime(
                expId_df[expId_column_name].str.split(split_char).str[date_position_in_expId], 
                format=date_format_in_expId).dt.strftime( final_date_format)
        return expId_df
        

In [39]:
current_tests = parse_exp_start_from_expId(current_tests)

In [43]:
for i in range(len(current_tests)):
    print(current_tests['experimentID'][i])
    ch_client.query_dataframe('''
    SELECT 
        *
    FROM rw.WEB_abtests_in
    WHERE date>='{current_tests['exp_start_date'][i]}' AND date<=yesterday()
        AND experimentID='{current_tests['experimentID'][i]}'
    ''')

dc_mobilemenu_20220831_231312889
dc_cookiesblock2_20220816_231312889
dc_deliverytimermobile_20220825_231312889
rko_testpagetarrifs_20220729_227240298
rko_vigetindentmb_20220816_227240298
dc_cookiesblockver3_20220817_231312889
rko_testcommunicationnul_20220803_227240298
dc_kukiblock_20220810_231312889
dc_ctaalfacard2_20220726_183996781
dc_deliverytimerdesktop_20220824_231312889
rko_vigetindent_20220816_227240298
invest_testcalculator3_20220712_226585591
dc_deliverytimermobile_20220824_231312889
dc_ctaalfacard_20220723_183996781
dc_namecta_20220729_183996781


In [48]:
test1 = ch_client.query_dataframe(f'''
    SELECT 
        *
    FROM rw.WEB_abtests_in
    WHERE date>='{current_tests['exp_start_date'][0]}' AND date<=yesterday()
        AND experimentID='{current_tests['experimentID'][0]}'
    ''')

In [47]:
from ab_test_pipeline import Pipeline

In [49]:
test1

,date,device,experimentID,experimentVariant,visitor_type,channelGroupB,ab_product,ab_name,ab_start_date,ab_goal_id,visits,goal_visits
0,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,0,New Visitor,01. бесплатный,dc,mobilemenu,20220831,231312889,753,82
1,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,1,Returning Visitor,13. crm,dc,mobilemenu,20220831,231312889,1,0
2,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,0,Returning Visitor,07. retention,dc,mobilemenu,20220831,231312889,55,3
3,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,0,Returning Visitor,04. контекст - сети,dc,mobilemenu,20220831,231312889,38,4
4,2022-09-04,tablet,dc_mobilemenu_20220831_231312889,1,New Visitor,05. платный соц. сети,dc,mobilemenu,20220831,231312889,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
426,2022-09-01,mobile,dc_mobilemenu_20220831_231312889,0,New Visitor,05. платный соц. сети,dc,mobilemenu,20220831,231312889,5,0
427,2022-09-01,tablet,dc_mobilemenu_20220831_231312889,1,Returning Visitor,04. контекст - сети,dc,mobilemenu,20220831,231312889,2,0
428,2022-09-01,mobile,dc_mobilemenu_20220831_231312889,0,Returning Visitor,12. офлайн активности,dc,mobilemenu,20220831,231312889,5,1
429,2022-09-01,desktop,dc_mobilemenu_20220831_231312889,1,Returning Visitor,01. бесплатный,dc,mobilemenu,20220831,231312889,7,0


In [ ]:
p = Pipeline(test1)
res1 = p.pipeline('date', experiment_var_col="experimentVariant", 
                groups = ['device', 'visitor_type', 'channelGroupB'],
                #metric_aggregations={"conversions":"sum", "sessions":"sum"},
               metrics_for_binary={"goal_visits": "visits"})[0]


In [58]:
test1.loc[0, ['experimentID', 'ab_product', 'ab_name', 'ab_start_date', 'ab_goal_id', 'date']]

experimentID     dc_mobilemenu_20220831_231312889
ab_product                                     dc
ab_name                                mobilemenu
ab_start_date                            20220831
ab_goal_id                              231312889
date                                   2022-09-04
Name: 0, dtype: object

In [59]:
res1[['experimentID', 'ab_product', 'ab_name', 
    'ab_start_date', 'ab_goal_id', 'date']] = test1.loc[0, 
            ['experimentID', 'ab_product', 'ab_name', 'ab_start_date', 'ab_goal_id', 'date']]

In [62]:
res1.drop('metric', axis='columns', inplace=True)
res1

,device,visitor_type,channelGroupB,first,second,mean_lift,test_type,p_value,experimentID,ab_product,ab_name,ab_start_date,ab_goal_id,date
0,No group,No group,No group,df_0,df_1,0.013246,z-test,0.533592,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
1,No group,No group,No group,df_0,df_1,-0.013149,bayes_test,0.266802,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
2,mobile,No group,No group,df_0,df_1,0.0148,z-test,0.495043,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
3,mobile,No group,No group,df_0,df_1,-0.014472,bayes_test,0.247526,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
4,tablet,No group,No group,df_0,df_1,-0.083245,z-test,0.897847,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,desktop,Returning Visitor,06. альфапартнерс,df_0,df_1,0.44186,bayes_test,0.646154,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
254,desktop,Returning Visitor,08. другие платные,df_1,df_0,inf,z-test,0.083265,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
255,desktop,Returning Visitor,08. другие платные,df_1,df_0,-1.666667,bayes_test,0.1,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04
256,desktop,Returning Visitor,02. контекст - брендовый поиск,df_1,df_0,NaN,z-test,NaN,dc_mobilemenu_20220831_231312889,dc,mobilemenu,20220831,231312889,2022-09-04


In [64]:
lst= res1.values.tolist()
for i in range(len(lst)):
    for j in range(len(res1.columns)):
        lst[i][j]=str(lst[i][j])

In [68]:
ch_client.execute('INSERT INTO rw.WEB_ab_tests_results VALUES', res1.values.tolist())

258

In [120]:
def query_from_ab_test_stats_clickhouse(clickhouse_client, table_name, expId, exp_start_date,
    expId_column_name='experimentID', date_column_name='date', date='yesterday()'):
    try:
        datetime.datetime.strptime(date, "%Y-%m-%d")
        return ch_client.query_dataframe(f'''
        SELECT 
            *
        FROM {table_name}
        WHERE {date_column_name}>='{exp_start_date}' AND {date_column_name}<='{date}'
            AND {expId_column_name}='{expId}'
        ''')
    except ValueError:    
        return ch_client.query_dataframe(f'''
        SELECT 
            *
        FROM {table_name}
        WHERE {date_column_name}>='{exp_start_date}' AND {date_column_name}<={date}
            AND {expId_column_name}='{expId}'
        ''')

In [85]:
query_from_ab_test_stats_clickhouse(ch_client, 'rw.WEB_abtests_in', 
            expId=current_tests['experimentID'][0], exp_start_date=current_tests['exp_start_date'][0],
            expId_column_name='experimentID', date_column_name='date', date='2022-09-05')

,date,device,experimentID,experimentVariant,visitor_type,channelGroupB,ab_product,ab_name,ab_start_date,ab_goal_id,visits,goal_visits
0,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,0,New Visitor,01. бесплатный,dc,mobilemenu,20220831,231312889,753,82
1,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,1,Returning Visitor,13. crm,dc,mobilemenu,20220831,231312889,1,0
2,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,0,Returning Visitor,07. retention,dc,mobilemenu,20220831,231312889,55,3
3,2022-09-04,mobile,dc_mobilemenu_20220831_231312889,0,Returning Visitor,04. контекст - сети,dc,mobilemenu,20220831,231312889,38,4
4,2022-09-04,tablet,dc_mobilemenu_20220831_231312889,1,New Visitor,05. платный соц. сети,dc,mobilemenu,20220831,231312889,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
426,2022-09-03,mobile,dc_mobilemenu_20220831_231312889,0,New Visitor,03. контекст - не брендовый поиск,dc,mobilemenu,20220831,231312889,17,2
427,2022-09-03,desktop,dc_mobilemenu_20220831_231312889,0,Returning Visitor,02. контекст - брендовый поиск,dc,mobilemenu,20220831,231312889,1,0
428,2022-09-03,mobile,dc_mobilemenu_20220831_231312889,0,New Visitor,06. альфапартнерс,dc,mobilemenu,20220831,231312889,1234,38
429,2022-09-03,mobile,dc_mobilemenu_20220831_231312889,1,Returning Visitor,02. контекст - брендовый поиск,dc,mobilemenu,20220831,231312889,534,40


In [111]:
def ab_test_scheduler(clickhouse_client, stats_table_name, results_table_name, date='yesterday()',
    groups=['device', 'visitor_type', 'channelGroupB'],
    conversions_column_name='goal_visits', sessions_column_name='visits',
    expId_column_name='experimentID', expVar_column_name='experimentVariant', 
    date_column_name='date'):
    current_tests = get_current_tests(clickhouse_client, stats_table_name, expId_column_name, date=date)
    print('Got current tests')

    for i in range(len(current_tests)):
        print('Iteration ', i)
        test_stats = query_from_ab_test_stats_clickhouse(clickhouse_client, stats_table_name, 
            expId=current_tests['experimentID'][i], exp_start_date=current_tests['exp_start_date'][i],
            expId_column_name=expId_column_name, date_column_name=date_column_name, date=date)
        print('Got test stats')
        
        p = Pipeline(test_stats)
        res = p.pipeline(date_column_name, experiment_var_col=expVar_column_name, 
                        groups = groups,
                        metrics_for_binary={conversions_column_name: sessions_column_name})[0]
        print('Got test results')
        
        res[['experimentID', 'ab_product', 'ab_name', 
            'ab_start_date', 'ab_goal_id', 'date']] = test_stats.loc[0, 
                    ['experimentID', 'ab_product', 'ab_name', 'ab_start_date', 'ab_goal_id', 'date']]
        res.drop('metric', axis='columns', inplace=True)
        
        clickhouse_client.execute(f'INSERT INTO {results_table_name} VALUES', res.values.tolist())
        print('Wrote results to CH')


In [ ]:
ab_test_scheduler(ch_client, 'rw.WEB_abtests_in', 'rw.WEB_ab_tests_results', date='2022-09-04')

In [117]:
import datetime
date_a = datetime.date(2022,8,29)
datetime.datetime.strftime(date_a,'%Y-%m-%d')

'2022-08-29'

In [ ]:
date_a = datetime.date(2022,8,30)
while date_a <= datetime.date(2022,9,3):
    ab_test_scheduler(ch_client, 'rw.WEB_abtests_in', 'rw.WEB_ab_tests_results', date=datetime.datetime.strftime(date_a,'%Y-%m-%d'))
    date_a+=datetime.timedelta(days=1)

In [1]:
import ab_test_scheduler

In [2]:
ab_test_scheduler.get_current_tests()

TypeError: get_current_tests() missing 2 required positional arguments: 'clickhouse_client' and 'table_name'